# CKAマトリクスの可視化

In [ ]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def plot_cka(csv_path, ax=None, cmap="coolwarm", vmin=0, vmax=1, title=None):
    """
    1つのCKA CSVファイルをヒートマップとして描画する関数。

    Parameters
    ----------
    csv_path : str
        CKA行列を格納したCSVファイルのパス
    ax : matplotlib.axes.Axes, optional
        描画先のAxes（指定しない場合は新規作成）
    cmap : str
        カラーマップ（例: "viridis", "coolwarm"）
    vmin, vmax : float
        カラースケールの範囲（CKA値に合わせて0〜1を指定するのが一般的）
    title : str
        サブタイトルやエポック番号など
    """
    # === CSVをDataFrameとして読み込み ===
    df = pd.read_csv(csv_path, header=None)
    
    # === 描画先のAxes設定 ===
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 5))
    
    # === ヒートマップ描画 ===
    sns.heatmap(df, ax=ax, cmap=cmap, vmin=vmin, vmax=vmax, cbar=False)
    
    # === 軸とタイトル ===
    ax.set_xlabel("Student Layer")
    ax.set_ylabel("Teacher Layer")
    if title:
        ax.set_title(title)
    else:
        ax.set_title(os.path.basename(csv_path))
    
    return ax


def plot_all_cka_in_folder(folder_path, max_cols=4):
    """
    指定フォルダ内のすべてのCKA CSVファイルをグリッド表示する関数。
    
    Parameters
    ----------
    folder_path : str
        CKA CSVファイル群があるディレクトリ
    max_cols : int
        一行あたりに並べるヒートマップの数
    """
    # === ファイル一覧取得 ===
    csv_files = sorted(glob.glob(os.path.join(folder_path, "cka_epoch_*.csv")))
    if not csv_files:
        print("No CSV files found in:", folder_path)
        return
    
    n = len(csv_files)
    ncols = min(max_cols, n)
    nrows = (n + ncols - 1) // ncols  # 行数を自動計算
    
    # === Figure作成 ===
    fig, axes = plt.subplots(nrows, ncols, figsize=(4*ncols, 4*nrows))
    axes = axes.flatten() if n > 1 else [axes]
    
    # === 各ファイルを順に描画 ===
    for i, csv_path in enumerate(csv_files):
        epoch_name = os.path.splitext(os.path.basename(csv_path))[0]
        plot_cka(csv_path, ax=axes[i], title=epoch_name)
    
    # === 余白調整 ===
    for j in range(i+1, len(axes)):
        axes[j].axis("off")  # 残りの空欄を非表示にする
    
    plt.tight_layout()
    plt.show()


In [ ]:
folder = os.path.join(
    "save", "cka_logs",
    "cka_log_S_vgg16_bn-T_vgg16_bn-cifar100-trial_0-epochs_240-bs_8-ckad-cls_1.0-div_1.0-beta_1.0-20251019_101628"
)
plot_all_cka_in_folder(folder)